# Activities Scraper

In [154]:
import pandas as pd
import re
import numpy as np

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [155]:
species = pd.read_csv('species.csv')

# add a park code column
species['Park Code'] = species['Species ID'].str[:4]
species['Park Name'] = species['Park Name'].map(lambda x: x.rstrip('and Preserve'))

# all np codes
park_codes = list(species['Park Code'].unique())

/var/folders/vb/bprs19k12d92r7xpmxnv1rx40000gn/T/ipykernel_28459/3062235437.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  species = pd.read_csv('species.csv')


In [156]:
driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))


### Testing different parks

In [157]:
# web driver link
driver.get("https://www.nps.gov/{}/planyourvisit/things2do.htm".format(park_codes[6]))

In [158]:
# web page title
driver.title

'Things To Do - Bryce Canyon National Park (U.S. National Park Service)'

In [159]:
#activities_web = driver.find_elements(By.XPATH, '//h2[@class="Feature-title carrot-end"]')
#activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
#activities_txt = [b.text for b in activities_web]

#activities_web = driver.find_elements(By.XPATH, '//h2[@class="Feature-title carrot-end"]')
#activities_txt = [b.text for b in activities_web]

# for bibe, glacier
#activities_web = driver.find_elements(By.XPATH, '//h3[@class="ActivityListing-title"]')
#activities_txt = [b.text for b in activities_web]

# for arches, canyonlands, congaree. everg, gb
#activities_links = driver.find_elements(By.CSS_SELECTOR, "a[id^=CP___PAGEID]")
#activities_txt = [b.text for b in activities_links]

#bisc has to click on outdoor activities

# for bryce

activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
activities_txt = [b.text for b in activities_web]

#activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
#activities_txt = [b.text for b in activities_web]
activities_txt

['See the Bryce Amphitheater',
 'Drive the Southern Scenic Drive',
 'See the Night Sky',
 'Get Your Junior Ranger Badge',
 'Day Hikes',
 'Ride the Shared-use Path',
 'Go Horseback Riding',
 'Walk the Rim Trail',
 'Explore the Visitor Center Museum',
 'View Wildlife']

### Pull activities for all parks

In [160]:
def pull_activities(code):
    driver.get("https://www.nps.gov/{}/planyourvisit/things2do.htm".format(code))
    activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
    activities_txt = [b.text for b in activities_web]


    if len(activities_txt) == 0:
        activities_web = driver.find_elements(By.XPATH, '//h2[@class="Feature-title carrot-end"]')
        activities_txt = [b.text for b in activities_web]
    if len(activities_txt) == 0:
        activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
        activities_txt = [b.text for b in activities_web]
    if len(activities_txt) == 0:
        activities_web = driver.find_elements(By.XPATH, '//h2[@class="Feature-title carrot-end"]')
        activities_txt = [b.text for b in activities_web]
    if len(activities_txt) == 0:
        # for bibe, glacier
        activities_web = driver.find_elements(By.XPATH, '//h3[@class="ActivityListing-title"]')
        activities_txt = [b.text for b in activities_web]
    if len(activities_txt) == 0:
        # for arches, canyonlands, congaree. everg, gb
        activities_links = driver.find_elements(By.CSS_SELECTOR, "a[id^=CP___PAGEID]")
        activities_txt = [b.text for b in activities_links]
        
        #bisc has to click on outdoor activities
    if len(activities_txt) == 0:
        # for bryce
        activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
        activities_txt = [b.text for b in activities_web]
    #if len(activities_txt) == 0:
    #    activities_web = driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
    #    activities_txt = [b.text for b in activities_web]
        
    return activities_txt

In [161]:
activities_df = pd.DataFrame()
activities_df['park'] = park_codes
activities_df['activities'] = [pull_activities(c) for c in activities_df['park']]
activities_df

,park,activities
0,ACAD,[Hike Sieur de Monts to Jesup Path with Island...
1,ARCH,"[Auto Touring, Backpacking, Biking, Camping, C..."
2,BADL,"[Visit the Ben Reifel Visitor Center, Visit th..."
3,BIBE,"[List of Hiking Trails in Big Bend, See the Ri..."
4,BISC,"[Guided Tours, Indoor Activities, For Kids]"
5,BLCA,[Half-Day at Black Canyon (Summer)]
6,BRCA,[]
7,CANY,"[Auto Touring, Backpacking, Biking, Boating, C..."
8,CARE,"[Road Tours, Camping, Hiking Capitol Reef, Out..."
9,CAVE,"[Enjoy the Cavern on Your Own, Visitor Center,..."


In [162]:
#prk = activities_df.iloc[2]
#driver.get("https://www.nps.gov/{}/planyourvisit/things2do.htm".format(prk.park))

In [163]:
#badl_activities= driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
#badl_txt = [b.text for b in badl_activities]


In [164]:
prk = activities_df.iloc[6]
driver.get("https://www.nps.gov/{}/planyourvisit/things2do.htm".format(prk.park))

In [165]:
brca_activities= driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
brca_txt = [b.text for b in brca_activities]

In [147]:
#prk = activities_df.iloc[51]
#driver.get("https://www.nps.gov/{}/planyourvisit/things2do.htm".format(prk.park))

In [148]:
#wica_activities= driver.find_elements(By.XPATH, '//h3[@class="ListingResults-title"]')
#wica_txt = [b.text for b in wica_activities]

In [149]:
#activities_df.iloc[2]['activities'] = badl_txt
activities_df.iloc[6]['activities'] = brca_txt
#activities_df.iloc[51]['activities'] = wica_txt

/var/folders/vb/bprs19k12d92r7xpmxnv1rx40000gn/T/ipykernel_28459/2511364448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df.iloc[2]['activities'] = badl_txt
/var/folders/vb/bprs19k12d92r7xpmxnv1rx40000gn/T/ipykernel_28459/2511364448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_df.iloc[6]['activities'] = brca_txt
/var/folders/vb/bprs19k12d92r7xpmxnv1rx40000gn/T/ipykernel_28459/2511364448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [150]:
activities_df

,park,activities
0,ACAD,[Hike Sieur de Monts to Jesup Path with Island...
1,ARCH,"[Auto Touring, Backpacking, Biking, Camping, C..."
2,BADL,"[Visit the Ben Reifel Visitor Center, Visit th..."
3,BIBE,"[List of Hiking Trails in Big Bend, See the Ri..."
4,BISC,"[Guided Tours, Indoor Activities, For Kids]"
5,BLCA,[Half-Day at Black Canyon (Summer)]
6,BRCA,[]
7,CANY,"[Auto Touring, Backpacking, Biking, Boating, C..."
8,CARE,"[Road Tours, Camping, Hiking Capitol Reef, Out..."
9,CAVE,"[Enjoy the Cavern on Your Own, Visitor Center,..."
